# MIIII

In [1]:
import jax.numpy as jnp
from jax import random
import jax
from oeis import oeis
import optax
from functools import partial
from sklearn.metrics import confusion_matrix, f1_score
from matplotlib import pyplot as plt
import src

In [6]:
def predict(apply_fn, params, rng, x):
    return (jax.nn.sigmoid(apply_fn(params, rng, x, 0)) > 0.5).astype(jnp.int32)

In [3]:
# config and init
conf, (rng, key) = src.get_conf(), random.split(random.PRNGKey(0))
data = src.prime_fn(conf.n, partial(src.base_n, conf.base))
params = src.init_fn(key, conf)

In [4]:
# train
apply_fn = src.make_apply_fn(src.vaswani_fn)
train_fn, opt_state = src.init_train(apply_fn, params, conf, *data)
(params, opt_state, rng), losses = train_fn(params, opt_state, rng, conf.epochs)

In [7]:
# evaluate
def make_plots(losses, conf, params):
    (train_x, train_y), (valid_x, valid_y) = data
    train_pred = predict(apply_fn, params, rng, train_x)
    valid_pred = predict(apply_fn, params, rng, valid_x)
    src.polar_plot(train_y, train_pred, conf, "train")
    src.polar_plot(valid_y, valid_pred, conf, "valid", offset=train_y.shape[0])
    src.curve_plot(jnp.log(losses), conf, params)


make_plots(losses, conf, params)

ValueError: vmap in_axes specification must be a tree prefix of the corresponding value, got specification (None, None, 0, None) for value tree PyTreeDef(({'blocks': [{'ffwd': (*, *, *, *), 'head': (*, *, *, *)}, {'ffwd': (*, *, *, *), 'head': (*, *, *, *)}], 'lm_head': *, 'pos_emb': *, 'tok_emb': *}, *, *)).